In [0]:
# Creating a SparkSession in Python
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("Word Count")\
    .config("spark.some.config.option", "some-value")\
    .getOrCreate()


In [0]:
df1 = spark.range(2, 10000000, 2)
df2 = spark.range(2, 10000000, 4)
step1 = df1.repartition(5)
step12 = df2.repartition(6)
step2 = step1.selectExpr("id * 5 as id")
step3 = step2.join(step12, ["id"])
step4 = step3.selectExpr("sum(id)")

step4.collect() # 2500000000000


Out[2]: [Row(sum(id)=2500000000000)]

In [0]:
step4.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
 *(5) HashAggregate(keys=[], functions=[finalmerge_sum(merge sum#58L) AS sum(id#51L)#54L])
 +- ShuffleQueryStage 3, Statistics(sizeInBytes=272.0 B, rowCount=17, isRuntime=true)
 +- Exchange SinglePartition, true, [id=#412]
 +- *(4) HashAggregate(keys=[], functions=[partial_sum(id#51L) AS sum#58L])
 +- *(4) Project [id#51L]
 +- *(4) SortMergeJoin [id#51L], [id#45L], Inner
 :- Sort [id#51L ASC NULLS FIRST], false, 0
 : +- CustomShuffleReader coalesced
 : +- ShuffleQueryStage 2, Statistics(sizeInBytes=76.3 MiB, rowCount=5.00E+6, isRuntime=true)
 : +- Exchange hashpartitioning(id#51L, 200), true, [id=#262]
 : +- *(3) Project [(id#43L * 5) AS id#51L]
 : +- ShuffleQueryStage 0, Statistics(sizeInBytes=76.3 MiB, rowCount=5.00E+6, isRuntime=true)
 : +- Exchange RoundRobinPartitioning(5), false, [id=#150]
 : +- *(1) Range (2, 10000000, step=2, splits=8)
 +- Sort [id#45L ASC NULLS FIRST], false, 0
 +- CustomShuffleReader coalesced
 +- ShuffleQueryStage 1, Statistics(sizeInBytes=38.1 MiB, rowCount=2.50E+6, isRuntime=true)
 +- Exchange hashpartitioning(id#45L, 200), true, [id=#156]
 +- *(2) Range (2, 10000000, step=4, splits=8)
+- == Initial Plan ==
 HashAggregate(keys=[], functions=[finalmerge_sum(merge sum#58L) AS sum(id#51L)#54L])
 +- Exchange SinglePartition, true, [id=#60]
 +- HashAggregate(keys=[], functions=[partial_sum(id#51L) AS sum#58L])
 +- Project [id#51L]
 +- SortMergeJoin [id#51L], [id#45L], Inner
 :- Sort [id#51L ASC NULLS FIRST], false, 0
 : +- Exchange hashpartitioning(id#51L, 200), true, [id=#52]
 : +- Project [(id#43L * 5) AS id#51L]
 : +- Exchange RoundRobinPartitioning(5), false, [id=#42]
 : +- Range (2, 10000000, step=2, splits=8)
 +- Sort [id#45L ASC NULLS FIRST], false, 0
 +- Exchange hashpartitioning(id#45L, 200), true, [id=#53]
 +- Range (2, 10000000, step=4, splits=8)